<a href="https://colab.research.google.com/github/sungjk1999/smart-factory-quality-classification/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


from google.colab import drive
drive.mount('/content/gdrive')


"""
PRODUCT_ID : 제품의 고유 ID

Y_Class : 제품 품질 상태(Target) 
    0 : 적정 기준 미달 (부적합) 
    1 : 적합 
    2 : 적정 기준 초과 (부적합)

Y_Quality : 제품 품질 관련 정량적 수치 

TIMESTAMP : 제품이 공정에 들어간 시각 

LINE : 제품이 들어간 공정 LINE 종류 ('T050304', 'T050307', 'T100304', 'T100306', 'T010306', 'T010305' 존재) 

PRODUCT_CODE : 제품의 CODE 번호 ('A_31', 'T_31', 'O_31' 존재) 

X_1 ~ X_2875 : 공정 과정에서 추출되어 비식별화된 변수
"""


dir_path = "/content/gdrive/MyDrive/Colab Notebooks/open"

train = pd.read_csv(f"{dir_path}/train.csv")

Mounted at /content/gdrive


In [3]:
# X_1 ~ X_2875 열의 std가 0 또는 nan인 경우를 제거
# test.csv 에서도 동일하게 적용해야함.

invalid = []

tmp = train.describe()

for col in tmp.columns:
    std = tmp.loc['std', col]
    if std == 0 or np.isnan(std):
        invalid.append(col)
        
train = train.drop(columns=invalid)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Columns: 2433 entries, PRODUCT_ID to X_2870
dtypes: float64(2428), int64(1), object(4)
memory usage: 11.1+ MB


In [15]:
# Y_Quality 값을 통해서, Y_Class를 예측하므로 어느 정도에서 끊기는지 확인
# 정량 데이터이므로, 구간이 겹치지 않는 것을 확인할 수 있음.

over = train[train['Y_Class'] == 2]['Y_Quality']
normal = train[train['Y_Class'] == 1]['Y_Quality']
under = train[train['Y_Class'] == 0]['Y_Quality']

print(f"적정 기준 초과 여부 커트라인 : [{normal.max()}, {over.min()}]")
print(f"적정 기준 미달 여부 커트라인 : [{under.max()}, {normal.min()}]")

적정 기준 초과 여부 커트라인 : [0.534842857, 0.534950794]
적정 기준 미달 여부 커트라인 : [0.525066667, 0.525085714]
